In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
dir='/content/drive/MyDrive/SpotifyMilionPlaylist/'

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

In [5]:
data_dir=dir+'data/'

In [12]:
def read_multiple_spotify_json(folder_path: str, max_files: int = 100) -> pd.DataFrame:
    """
    Legge e concatena i primi `max_files` file JSON del Spotify Million Playlist Dataset in un unico DataFrame.

    :param folder_path: Percorso della cartella contenente i file JSON.
    :param max_files: Numero massimo di file da leggere (default: 100).
    :return: DataFrame contenente tutte le playlist e le tracce.
    """
    all_playlists = []

    # Prendi i primi 100 file JSON dalla cartella
    json_files = sorted([f for f in os.listdir(folder_path) if f.endswith(".json")])[:max_files]

    for json_file in tqdm(json_files, desc="Reading JSON files"):
        file_path = os.path.join(folder_path, json_file)

        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # Estrarre informazioni da ogni playlist
            for playlist in data['playlists']:
                playlist_id = playlist.get('pid')  # ID della playlist
                playlist_name = playlist.get('name', 'Unknown')  # Nome della playlist
                num_tracks = playlist.get('num_tracks', 0)  # Numero di tracce

                # Estrarre tracce
                for track in playlist.get('tracks', []):
                    all_playlists.append({
                        'playlist_id': playlist_id,
                        'track_name': track.get('track_name', 'Unknown'),
                        'artist_name': track.get('artist_name', 'Unknown'),
                        'album_name': track.get('album_name', 'Unknown'),
                        'track_uri': track.get('track_uri', 'Unknown')
                    })

        except Exception as e:
            print(f"Errore durante la lettura del file {json_file}: {e}")

    # Creare il DataFrame concatenando tutti i dati
    return pd.DataFrame(all_playlists)



In [13]:
df = read_multiple_spotify_json(data_dir, max_files=150)

Reading JSON files: 100%|██████████| 150/150 [01:33<00:00,  1.60it/s]


In [14]:
df.head()

,playlist_id,track_name,artist_name,album_name,track_uri
0,0,Lose Control (feat. Ciara & Fat Man Scoop),Missy Elliott,The Cookbook,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI
1,0,Toxic,Britney Spears,In The Zone,spotify:track:6I9VzXrHxO9rA9A5euc8Ak
2,0,Crazy In Love,Beyoncé,Dangerously In Love (Alben für die Ewigkeit),spotify:track:0WqIKmW4BTrj3eJFmnCKMv
3,0,Rock Your Body,Justin Timberlake,Justified,spotify:track:1AWQoqb9bSvzTjaLralEkT
4,0,It Wasn't Me,Shaggy,Hot Shot,spotify:track:1lzr43nnXAijIGYnCT8M8H


In [15]:
def get_cooccurrence_similarity(track_name, df, top_n=10):
    """
    Trova le canzoni più simili a una traccia in base alla co-presenza nelle playlist,
    cercando internamente per track_uri invece di track_name.

    :param track_name: Nome della traccia di riferimento.
    :param df: DataFrame con tutte le tracce e playlist.
    :param top_n: Numero di canzoni simili da restituire.
    :return: Lista con i nomi delle tracce più simili.
    """
    # Troviamo il track_uri corrispondente al nome della traccia
    track_uris = df[df['track_name'].str.lower() == track_name.lower()]['track_uri'].unique()

    if len(track_uris) == 0:
        print(f"La traccia '{track_name}' non è presente nel dataset.")
        return None

    track_uri = track_uris[0]  # Se ci sono più URI, prendiamo il primo

    # Troviamo le playlist che contengono questa traccia
    playlists_with_track = df[df['track_uri'] == track_uri]['playlist_id'].unique()

    if len(playlists_with_track) == 0:
        print(f"Nessuna playlist trovata con la traccia '{track_name}'.")
        return None

    # Filtriamo il dataset per prendere solo le tracce di quelle playlist
    cooccurring_tracks = df[df['playlist_id'].isin(playlists_with_track)]

    # Contiamo la frequenza di ogni traccia presente in queste playlist (escludendo la traccia di input)
    track_counts = cooccurring_tracks['track_uri'].value_counts().reset_index()
    track_counts.columns = ['track_uri', 'cooccurrence_score']

    # Rimuoviamo la canzone di input dalla lista
    track_counts = track_counts[track_counts['track_uri'] != track_uri]

    # Aggiungiamo i nomi delle tracce
    track_counts = track_counts.merge(df[['track_uri', 'track_name']].drop_duplicates(), on='track_uri', how='left')

    # Restituiamo solo i nomi delle tracce più simili
    return track_counts['track_name'].head(top_n).tolist()


In [21]:
track_name = "Toxic"  # Sostituisci con una traccia a scelta
df_similar = get_cooccurrence_similarity(track_name, df, top_n=5)

df_similar

['Hollaback Girl', 'Yeah!', "Hips Don't Lie", 'Crazy In Love', 'SexyBack']

In [43]:
df['combined_features'] = df['track_name'] + " " + df['artist_name'] + " " + df['album_name']

In [44]:
tfidf = TfidfVectorizer(stop_words="english")

tfidf_matrix = tfidf.fit_transform(df['combined_features'])

In [37]:
def get_similar_songs_per_track(track_name, df, tfidf_matrix, top_n=10):
    """
    Trova le canzoni più simili a una traccia data, escludendo duplicati e risultati identici.

    :param track_name: Nome della traccia di riferimento.
    :param df: DataFrame con il dataset delle tracce.
    :param tfidf_matrix: Matrice TF-IDF delle tracce.
    :param top_n: Numero di canzoni simili da restituire.
    :return: DataFrame con le canzoni più simili.
    """
    track_idx = df[df['track_name'].str.lower() == track_name.lower()].index

    if len(track_idx) == 0:
        print(f"La traccia '{track_name}' non è presente nel dataset.")
        return None

    track_idx = track_idx[0]  # Ottieni il primo valore se esistono duplicati

    # Calcola la similarità coseno tra la traccia selezionata e tutto il dataset
    sim_scores = cosine_similarity(tfidf_matrix[track_idx], tfidf_matrix).flatten()

    # Escludiamo la traccia stessa e valori duplicati
    sim_indices = np.where((sim_scores < 1.0))[0]  # Escludiamo similarità 1.0
    sorted_indices = sim_indices[np.argsort(sim_scores[sim_indices])][-top_n*2:][::-1]  # Prendiamo più risultati per gestire duplicati

    # Creiamo il DataFrame con i risultati e rimuoviamo i duplicati
    similar_tracks = df.iloc[sorted_indices][['track_name', 'artist_name', 'album_name']].copy()
    similar_tracks['similarity_score'] = sim_scores[sorted_indices]
    similar_tracks = similar_tracks.drop_duplicates(subset=['track_name', 'artist_name']).head(top_n)  # Rimuove i duplicati

    return similar_tracks






In [1]:
get_similar_songs_per_track('Toxic', df, tfidf_matrix, top_n=5)

NameError: name 'get_similar_songs_per_track' is not defined

In [23]:
def get_similar_songs_for_playlist(playlist_id, df, tfidf_matrix, top_n=10):
    """
    Trova le canzoni più simili per tutte le tracce di una playlist.

    :param playlist_id: ID della playlist da analizzare.
    :param df: DataFrame con il dataset.
    :param tfidf_matrix: Matrice TF-IDF delle tracce.
    :param top_n: Numero di canzoni simili per ogni traccia della playlist.
    :return: DataFrame con le raccomandazioni.
    """
    # Seleziona le tracce della playlist
    playlist_tracks = df[df['playlist_id'] == playlist_id]

    recommended_tracks = pd.DataFrame()

    for track_name in playlist_tracks['track_name']:
        similar_tracks = get_similar_songs_per_track(track_name, df, tfidf_matrix, top_n)

        if similar_tracks is not None:
            recommended_tracks = pd.concat([recommended_tracks, similar_tracks])

    # Rimuoviamo duplicati e ordiniamo per similarità media
    recommended_tracks = recommended_tracks.groupby(['track_name', 'artist_name', 'album_name']).mean().reset_index()
    recommended_tracks = recommended_tracks.sort_values(by='similarity_score', ascending=False).head(top_n)

    return recommended_tracks
